# WISDEM Tutorial 2

In this tutorial, we will explore using some of OpenMDAO's MDAO capabilities with our relatively simple wind energy system model (the NREL cost and Scaling model).

First navigate to your folder with your WISDEM code.

In [125]:
cd C:\SystemsEng\WISDEM Tutorial\WISDEM\Code Files 2

C:\SystemsEng\WISDEM Tutorial\WISDEM\Code Files 2


The first step again is to import the WISDEM lcoe_csm_assembly model and initialize it with a set of turbine and project level parameters.

In [126]:
from openmdao.api import IndepVarComp, Component, Problem, Group
from fused_wind import create_interface , FUSED_Object , FUSED_OpenMDAO , set_output, set_input, fusedvar

# NREL cost and scaling model sub-assemblies
from nrel_csm_tcc import tcc_csm_fused
from nrel_csm_bos import bos_csm_fused
from nrel_csm_opex  import opex_csm_fused
from nrel_csm_fin import fin_csm_fused
from nrel_csm_aep import aep_csm_fused

import numpy as np

# openmdao example of execution
root = Group()
root.add('desvars',IndepVarComp([('machine_rating',5000.0),
                                                                 ('rotor_diameter', 126.0),
                                                                 ('hub_height', 90.0),
                                                                 ('turbine_number', 100.0),
                                                                 ('year', 2009.0),
                                                                 ('month',12.0),
                                                                 ]),promotes=['*'])
root.add('bos_csm_test', FUSED_OpenMDAO(bos_csm_fused()), promotes=['*'])
root.add('tcc_csm_test', FUSED_OpenMDAO(tcc_csm_fused()), promotes=['*'])
root.add('fin_csm_test', FUSED_OpenMDAO(fin_csm_fused()), promotes=['*'])
root.add('bos_opex_test', FUSED_OpenMDAO(opex_csm_fused()), promotes=['*'])
root.add('aep_test', FUSED_OpenMDAO(aep_csm_fused()), promotes=['*'])
# Tell the whole model to finite difference
root.deriv_options['type'] = 'fd'

prob = Problem(root)
prob.setup()


# set inputs
# simple test of module
# Turbine inputs
prob['rotor_diameter'] = 126.0
prob['blade_number'] = 3
prob['hub_height'] = 90.0    
prob['machine_rating'] = 5000.0

# Rotor force calculations for nacelle inputs
maxTipSpd = 80.0
maxEfficiency = 0.90201
ratedWindSpd = 11.5064
thrustCoeff = 0.50
airDensity = 1.225

ratedHubPower  = prob['machine_rating'] / maxEfficiency 
rotorSpeed     = (maxTipSpd/(0.5*prob['rotor_diameter'])) * (60.0 / (2*np.pi))
prob['rotor_thrust']  = airDensity * thrustCoeff * np.pi * prob['rotor_diameter']**2 * (ratedWindSpd**2) / 8
prob['rotor_torque'] = ratedHubPower/(rotorSpeed*(np.pi/30))*1000

prob['year'] = 2009
prob['month'] = 12

# AEP inputs
prob['max_tip_speed'] = 80.0 #Float(units = 'm/s', iotype='in', desc= 'maximum allowable tip speed for the rotor')
prob['max_power_coefficient'] = 0.488 #Float(iotype='in', desc= 'maximum power coefficient of rotor for operation in region 2')
prob['opt_tsr'] = 7.525 #Float(iotype='in', desc= 'optimum tip speed ratio for operation in region 2')
prob['cut_in_wind_speed'] = 3.0 #Float(units = 'm/s', iotype='in', desc= 'cut in wind speed for the wind turbine')
prob['cut_out_wind_speed'] = 25.0 #Float(units = 'm/s', iotype='in', desc= 'cut out wind speed for the wind turbine')
prob['altitude'] = 0.0 #Float(units = 'm', iotype='in', desc= 'altitude of wind plant')
prob['air_density'] = 0 #Float(units = 'kg / (m * m * m)', iotype='in', desc= 'air density at wind plant site')  # default air density value is 0.0 - forces aero csm to calculate air density in model
prob['max_efficiency'] = 0.902 #Float(iotype='in', desc = 'maximum efficiency of rotor and drivetrain - at rated power')
prob['thrust_coefficient'] = 0.5 #Float(iotype='in', desc='thrust coefficient at rated power')
prob['soiling_losses'] = 0.0
prob['array_losses'] = 0.1
prob['availability'] = 0.941
prob['turbine_number'] = 100
prob['shear_exponent'] = 0.1
prob['wind_speed_50m'] = 8.02
prob['weibull_k']= 2.15

# Finance, BOS and OPEX inputs
prob['RNA_mass'] = 256634.5 # RNA mass is not used in this simple model
prob['sea_depth'] = 20.0
prob['multiplier'] = 1.0


##############################################
Setup: Checking root problem for potential issues...

No recorders have been specified, so no data will be saved.

The following parameters have no associated unknowns:
RNA_mass
air_density
altitude
array_losses
availability
blade_number
cut_in_wind_speed
cut_out_wind_speed
max_efficiency
max_power_coefficient
max_tip_speed
multiplier
opt_tsr
sea_depth
shear_exponent
soiling_losses
thrust_coefficient
weibull_k
wind_speed_50m

Setup: Check of root problem complete.
##############################################



Let's just make sure everything runs properly by running the model and comparing the results to when we ran them before.

In [127]:
prob.run()
print("Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines")
for io in root.unknowns:
    print(io + ' ' + str(root.unknowns[io]))

Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines
machine_rating 5000.0
rotor_diameter 126.0
hub_height 90.0
turbine_number 100.0
year 2009.0
month 12.0
bos_costs 764764734.402
bos_breakdown_development_costs 19816413.6622
bos_breakdown_preparation_and_staging_costs 14155377.3927
bos_breakdown_transportation_costs 149852024.568
bos_breakdown_foundation_and_substructure_costs 212330660.89
bos_breakdown_electrical_costs 214524224.858
bos_breakdown_assembly_and_installation_costs 70776886.9634
bos_breakdown_soft_costs 0.0
bos_breakdown_other_costs 83309146.0675
turbine_cost 5383539.544
rotor_cost 1133729.26814
rotor_mass 87595.9193358
turbine_mass 747093.461862
coe 0.120210380123
lcoe 0.112342941751
avg_annual_opex 45280723.0581
opex_breakdown_preventative_opex 34279801.5634
opex_breakdown_corrective_opex 9104838.70968
opex_breakdown_lease_opex 1896082.78501
opex_breakdown_other_opex 0.0
net_aep 1600128912.66
rated_wind_speed 11.5064484118
rated_rotor_speed 12.

The results are the same so we are ready to test out an optimization.  Let's return to our land-based site in the US.  We found an improved LCOE when we went from our 5 MW Class 1 turbine in a high-wind speed site to a 2 MW Class III turbine in a low-wind speed site.  But, was that 2.3 MW Siemens turbine the best turbine for the site?  

With this simple model, we could easily do some parameter sweeps over the key turbine variables to find out.  But even varying 3 or 4 parameters at once (e.g. hub height, rotor diameter, and rated power) could be tedious and involve a large design space.  Let's try an optimization to speed things up a bit.



###### Optimal turbine selection for a given wind site.

First, let's set up the site conditions to be the same as before for the low-wind speed site from the first tutorial; and let's start with our SWT-2.3-120 turbine from before as well.

In [128]:
# 2 MW turbine specifications and plant attributes
# Turbine inputs
prob['rotor_diameter'] = 120.0
prob['blade_number'] = 3
prob['hub_height'] = 90.0    
prob['machine_rating'] = 2300.0

# Rotor force calculations for nacelle inputs
maxTipSpd = 80.0
maxEfficiency = 0.90201
ratedWindSpd = 11.5064
thrustCoeff = 0.50
airDensity = 1.225

ratedHubPower  = prob['machine_rating'] / maxEfficiency 
rotorSpeed     = (maxTipSpd/(0.5*prob['rotor_diameter'])) * (60.0 / (2*np.pi))
prob['rotor_thrust']  = airDensity * thrustCoeff * np.pi * prob['rotor_diameter']**2 * (ratedWindSpd**2) / 8
prob['rotor_torque'] = ratedHubPower/(rotorSpeed*(np.pi/30))*1000

# AEP inputs
prob['max_tip_speed'] = 80.0 #Float(units = 'm/s', iotype='in', desc= 'maximum allowable tip speed for the rotor')
prob['availability'] = 0.97
prob['turbine_number'] = 100
prob['shear_exponent'] = 0.143
prob['wind_speed_50m'] = 6.5
prob['weibull_k']= 2.1
prob['sea_depth']=0.0

And let's re-run the analysis again, just to double check:

In [129]:
prob.run()

print("Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines")
for io in root.unknowns:
    print(io + ' ' + str(root.unknowns[io]))

Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines
machine_rating 2300.0
rotor_diameter 120.0
hub_height 90.0
turbine_number 100.0
year 2009.0
month 12.0
bos_costs 94854288.9329
bos_breakdown_development_costs 5702191.69485
bos_breakdown_preparation_and_staging_costs 17074041.061
bos_breakdown_transportation_costs 13679987.6115
bos_breakdown_foundation_and_substructure_costs 11610853.9685
bos_breakdown_electrical_costs 31484296.7453
bos_breakdown_assembly_and_installation_costs 15302917.8518
bos_breakdown_soft_costs 0.0
bos_breakdown_other_costs 0.0
turbine_cost 3628565.51824
rotor_cost 974579.402682
rotor_mass 78322.9612083
turbine_mass 612421.926102
coe 0.0758879270859
lcoe 0.067897456315
avg_annual_opex 9554082.66673
opex_breakdown_preventative_opex 5993285.70499
opex_breakdown_corrective_opex 2636118.59583
opex_breakdown_lease_opex 924678.365913
opex_breakdown_other_opex 0.0
net_aep 799306988.873
rated_wind_speed 9.32728001239
rated_rotor_speed 12.7323954

Everything checks out!

Now on to the optimization.  Firstly, we need to replace the driver that the LCOE model was using (the basic driver for any assembly just runs the assembly once) with an optimizer.  OpenMDAO has several built-in optimizers; there is also a "pyOpt" plug-in for OpenMDAO which gives you access to all the optimization algorithms in pyOpt (website).  To install pyOpt, (installation instructions).

Let's use PyOpt's Cobyla optimizer for this analysis (describe cobyla).

To use the Cobyla optimizer, we replace the driver with the pyOptDriver and specify the optimizer as Cobyla.  We can also set various options for the optimizer; the options are documented in the pyOpt documentation (website).

In [130]:
from openmdao.api import ScipyOptimizer

prob.driver = ScipyOptimizer()
prob.driver.options['optimizer'] = 'COBYLA'

Now we have to add our objective and design variables.  In this case, our overall goal is finding the lowest LCOE for the site by selecting the turbine that is the most appropriate.

For a real site, this would of course involve a discrete choice by selecting the best turbine for the site from a set.  However, in this case let's consider an ideal world where we can continously vary the key turbine configuration parameters to get the best possible turbine and lowest possible LCOE.  Let's focus on the key turbine parameters of rated power, rotor diameter and hub height since these are the key inputs to this particular model.  We will keep the geared turbine as before and just vary these three turbine parameters.

Let's add these parameters and objectives:

In [131]:
# --- Objective ---
prob.driver.add_objective('coe') # / 1e-3

# --- Parameters ---
prob.driver.add_desvar('rotor_diameter', lower=80.0, upper=150.0)
prob.driver.add_desvar('machine_rating', lower=1000.0, upper=10000.0)
prob.driver.add_desvar('hub_height', lower=70.0, upper=120.0)


Normally we would then add constraints to the simulation; the upper and lower bounds on the design variables each act as a constraint so there are 6 total constraints in this optimization.  This model is vary simple with a small number of parameters so we will not add any further constraints.  However, as we will see, the accuracy of the results will be questionable.

Now let's try to run the optimization:

In [132]:
prob.setup()

# Set design variables
prob['rotor_diameter'] = 120.0
prob['machine_rating'] = 2300.0
prob['hub_height'] = 90.0

# set other input variables
# simple test of module
# Turbine inputs
prob['blade_number'] = 3

# Rotor force calculations for nacelle inputs
maxTipSpd = 80.0
maxEfficiency = 0.90201
ratedWindSpd = 11.5064
thrustCoeff = 0.50
airDensity = 1.225

ratedHubPower  = prob['machine_rating'] / maxEfficiency 
rotorSpeed     = (maxTipSpd/(0.5*prob['rotor_diameter'])) * (60.0 / (2*np.pi))
prob['rotor_thrust']  = airDensity * thrustCoeff * np.pi * prob['rotor_diameter']**2 * (ratedWindSpd**2) / 8
prob['rotor_torque'] = ratedHubPower/(rotorSpeed*(np.pi/30))*1000

prob['year'] = 2009
prob['month'] = 12

# AEP inputs
prob['max_tip_speed'] = 80.0 #Float(units = 'm/s', iotype='in', desc= 'maximum allowable tip speed for the rotor')
prob['max_power_coefficient'] = 0.488 #Float(iotype='in', desc= 'maximum power coefficient of rotor for operation in region 2')
prob['opt_tsr'] = 7.525 #Float(iotype='in', desc= 'optimum tip speed ratio for operation in region 2')
prob['cut_in_wind_speed'] = 3.0 #Float(units = 'm/s', iotype='in', desc= 'cut in wind speed for the wind turbine')
prob['cut_out_wind_speed'] = 25.0 #Float(units = 'm/s', iotype='in', desc= 'cut out wind speed for the wind turbine')
prob['altitude'] = 0.0 #Float(units = 'm', iotype='in', desc= 'altitude of wind plant')
prob['air_density'] = 0 #Float(units = 'kg / (m * m * m)', iotype='in', desc= 'air density at wind plant site')  # default air density value is 0.0 - forces aero csm to calculate air density in model
prob['max_efficiency'] = 0.902 #Float(iotype='in', desc = 'maximum efficiency of rotor and drivetrain - at rated power')
prob['thrust_coefficient'] = 0.5 #Float(iotype='in', desc='thrust coefficient at rated power')
prob['soiling_losses'] = 0.0
prob['array_losses'] = 0.1
prob['availability'] = 0.941
prob['turbine_number'] = 100
prob['shear_exponent'] = 0.1
prob['wind_speed_50m'] = 10.02
prob['weibull_k']= 2.15

# Finance, BOS and OPEX inputs
prob['RNA_mass'] = 256634.5 # RNA mass is not used in this simple model
prob['sea_depth'] = 20.0
prob['multiplier'] = 1.0


print("Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines")
for io in root.params:
    print(io + ' ' + str(root.params[io]))

prob.run()

##############################################
Setup: Checking root problem for potential issues...

No recorders have been specified, so no data will be saved.

The following parameters have no associated unknowns:
RNA_mass
air_density
altitude
array_losses
availability
blade_number
cut_in_wind_speed
cut_out_wind_speed
max_efficiency
max_power_coefficient
max_tip_speed
multiplier
opt_tsr
sea_depth
shear_exponent
soiling_losses
thrust_coefficient
weibull_k
wind_speed_50m

Setup: Check of root problem complete.
##############################################

Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines
aep_test.machine_rating 0.0
aep_test.rotor_diameter 0.0
aep_test.hub_height 0.0
aep_test.turbine_number 0.0
bos_opex_test.machine_rating 0.0
bos_opex_test.turbine_number 0.0
bos_opex_test.year 0.0
bos_opex_test.month 0.0
bos_opex_test.net_aep 0.0
tcc_csm_test.machine_rating 0.0
tcc_csm_test.rotor_diameter 0.0
tcc_csm_test.hub_height 0.0
tcc_csm_test.rotor_

The results should show that the rotor diameter, hub height and rated power have changed and the overall lcoe is lower than before.

Try adjusting the site mean wind speed (variable wind_speed_50m) and look at how this affects the optimization results.  What do you notice?  Run the optimization for mean wind speeds from 4 to 12 m/s and look at how the design variables and the objective function factors (AEP, TCC, etc) change with each optimization run.

In [133]:
print("Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines")
for io in root.unknowns:
    print(io + ' ' + str(root.unknowns[io]))

Overall cost of energy for an offshore wind plant with 100 NREL 5 MW turbines
machine_rating 2299.00265273
rotor_diameter 89.1495917804
hub_height 78.3112476202
turbine_number 100.0
year 2009.0
month 12.0
net_aep 962709881.308
rated_wind_speed 11.1464596671
rated_rotor_speed 17.1384683112
rotor_thrust 235590.575933
rotor_torque 1420143.76381
gross_aep 1136745638.57
capacity_factor 47.8026365679
avg_annual_opex 25951456.88
opex_breakdown_preventative_opex 20624278.1024
opex_breakdown_corrective_opex 4186409.66925
opex_breakdown_lease_opex 1140769.10833
opex_breakdown_other_opex 0.0
turbine_cost 2245913.20146
rotor_cost 407229.651926
rotor_mass 40896.2694865
turbine_mass 319310.692864
bos_costs 297632574.124
bos_breakdown_development_costs 9111597.51542
bos_breakdown_preparation_and_staging_costs 6508650.03524
bos_breakdown_transportation_costs 13664836.2083
bos_breakdown_foundation_and_substructure_costs 97629750.5286
bos_breakdown_electrical_costs 98638352.4047
bos_breakdown_assembly_a